In [20]:
from pulp import *


In [21]:
# Define workers and their hourly rates
workers = {
    'ProjectManager': 72,  # Example hourly rate for the project manager
    'FrontendDeveloper': 60,
    'BackendDeveloper': 60,
    'DataScientist': 58,
    'DataEngineer': 72,
}

In [22]:
# Define tasks and their estimated times for different scenarios
activities = {
    'Describe product': {'WorstCaseHours': 6},
    'Develop marketing strategy': {'WorstCaseHours': 16},
    'Design brochure': {'WorstCaseHours': 14},
    #'Develop product protoype': {'WorstCaseHours': 80},
    'Requirements analysis': {'WorstCaseHours': 10},
    'Software design': {'WorstCaseHours': 16},
    'System design':{'WorstCaseHours': 16}, 
    'Coding': {'WorstCaseHours': 40},
    'Write documentation': {'WorstCaseHours': 20},
    'Unit testing': {'WorstCaseHours': 20},
    'System testing': {'WorstCaseHours': 24},
    'Package deliverables': {'WorstCaseHours': 8},
    'Survey potential market': {'WorstCaseHours': 20},
    'Develop pricing plan': {'WorstCaseHours': 12},
    'Develop implementation  plan': {'WorstCaseHours': 16},
    'Write client proposal': {'WorstCaseHours': 8}

}


In [23]:
# Create a list of the activities
activities_list = list(activities.keys())


In [24]:
# Create a dictionary of the activity precedences
precedences = {
            'Describe product': [],
            'Develop marketing strategy': [], 
            'Design brochure': ['Describe product'], 
            #'Develop product protoype': [],  
            'Requirements analysis': ['Describe product'], 
            'Software design': ['Requirements analysis'], 
            'System design': ['Requirements analysis'], 
            'Coding': ['Software design','System design'],
            'Write documentation': ['Coding'],
            'Unit testing': ['Coding'],
            'System testing' : ['Unit testing'],
            'Package deliverables': ['Write documentation','System testing'],
            'Survey potential market':['Develop marketing strategy','Design brochure'],
            'Develop pricing plan': ['Package deliverables','Survey potential market'],
            'Develop implementation  plan': ['Describe product','Package deliverables'],
            'Write client proposal': ['Develop pricing plan','Develop implementation  plan']
            }


In [25]:
# Define decision variables
x = pulp.LpVariable.dicts("Hours", [(worker, task) for worker in workers for task in activities],
                           lowBound=0)


In [26]:
# Create the LP problem
prob = LpProblem("Critical Path", LpMinimize)


In [27]:
# Create the LP variables
start_times = {activity: LpVariable(f"start_{activity}", 0, None) for activity in activities_list}
end_times = {activity: LpVariable(f"end_{activity}", 0, None) for activity in activities_list}


In [28]:
# Add the constraints
for activity in activities_list:
    prob += end_times[activity] == start_times[activity] + activities[activity], f"{activity}_duration"
    for predecessor in precedences[activity]:
        prob += start_times[activity] >= end_times[predecessor], f"{activity}_predecessor_{predecessor}"


In [29]:
# Set the objective function
prob += lpSum([end_times[activity] for activity in activities_list]), "minimize_end_times"

# Solve the LP problem
status = prob.solve()

# Print the results
print("Critical Path time:")
for activity in activities_list:
    if value(start_times[activity]) == 0:
        print(f"{activity} starts at time 0")
    if value(end_times[activity]) == max([value(end_times[activity]) for activity in activities_list]):
        print(f"{activity} ends at {value(end_times[activity])} hours in duration")

# Print solution
print("\nSolution variable values:")
for var in prob.variables():
    if var.name != "_dummy":
        print(var.name, "=", var.varValue)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/Jai/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/0622698e1ad64603abd086e0a58f7d15-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/mg/gn4s4j_94_j8j560qd6pnnzm0000gp/T/0622698e1ad64603abd086e0a58f7d15-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 39 COLUMNS
At line 123 RHS
At line 158 BOUNDS
At line 159 ENDATA
Problem MODEL has 34 rows, 30 columns and 68 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 0 (-34) rows, 0 (-30) columns and 0 (-68) elements
Empty problem - 0 rows, 0 columns and 0 elements
Optimal - objective value 1082
After Postsolve, objective 1082, infeasibilities - dual 0 (0), primal 0 (0)
Optimal objective 1082 - 0 iterations time 0.002, Presolve 0.00
Option for printingOptions chang